# NIMS-KISTI-NVIDIA Hackathon 

# Problem Statement 



## Agenda 
 - Introduction to FourCastNet
 - Configure FourCastNet
 - Typoon Dataset (JMA best track) 
 - ECMWF ERA5 dataset(CDS API)
 - custom interval 
 - inference 
 - post processing 
 






## Introduction to FourCastNet

 - [FourCastNet Paper Link](https://arxiv.org/abs/2202.11214)
 - [GitHub Repository](https://github.com/NVlabs/FourCastNet)
 
FourCastNet, short for Fourier ForeCasting Neural Network, is a global data-driven weather forecasting model that provides accurate short to medium range global predictions at 0.25° resolution.
<div align="center">

<img src="https://github.com/NVlabs/FourCastNet/blob/master/assets/FourCastNet.gif?raw=true">
</div>





### FourCastNet Architecture :
FourCastNet uses a Fourier transform-based token-mixing scheme with a vision transformer (ViT) backbone. This approach is based on the recent Fourier neural operator that learns in a resolution-invariant manner and has shown success in modeling challenging partial differential equations (PDE) such as fluid dynamics.


<div align="center">
<img src="https://docscontent.nvidia.com/dims4/default/9c9b4d6/2147483647/strip/true/crop/1188x788+0+0/resize/1188x788!/quality/90/?url=https%3A%2F%2Fk3-prod-nvidia-docs.s3.amazonaws.com%2Fbrightspot%2Fsphinx%2F00000187-c443-dd05-a3bf-e57fb6480000%2Fdeeplearning%2Fmodulus%2Fmodulus-sym%2F_images%2Ffourcastnet_overview.png" >

</div>

 


### FourCastNet modeled variables:    
| Vertical Level | Variable                |
|:-------------- |:-----------------------|
| Surface        | U10, V10, T2M, SP, MSLP |
| 1000 hPa       | U, V, Z                |
| 850 hPa        | T, U, V, Z, RH         |
| 500 hPa        | T, U, V, Z, RH         |
| 50 hPa         | Z                      |
| Integrated     | TCWV                   |
 


## copy model source code 

### install required modules for FourCastNet

In [ ]:
! pip install wandb h5py mpi4py netCDF4 cdsapi  ruamel.yaml tqdm timm einops 
! pip install git+https://github.com/romerojosh/benchy.git
! pip install cdsapi netcdf4 xarray

### copy FourCastNet source code 

In [ ]:
!git clone https://github.com/NVlabs/FourCastNet.git

## download checkpoint and stat0

The model weights hosted at Trained Model Weights :

```
FCN_weights_v0/
│   backbone.ckpt  
│   precip.ckpt  
```

The pre-computed normalization statistics hosted at additional. It is crucial that you use the statistics that are provided if you are using the pre-trained model weights that we have provided since these stats were used when trainig the model. The normalization statistics go hand-in-hand with the trained model weights. The stats folder contains:


```
stats_v0
│   global_means.npy  
│   global_stds.npy  
│   land_sea_mask.npy  
│   latitude.npy  
│   longitude.npy  
│   time_means.npy
│   time_means_daily.h5
```


In [ ]:
%%time 
!wget -N -P FCN_weights_v0          https://portal.nersc.gov/project/m4134/FCN_weights_v0/backbone.ckpt 
#!wget -N -P FCN_weights_v0         https://portal.nersc.gov/project/m4134/FCN_weights_v0/precip.ckpt               
!wget -N -P FCN_weights_v0/stats_v0 https://portal.nersc.gov/project/m4134/FCN_weights_v0/stats_v0/global_means.npy 
!wget -N -P FCN_weights_v0/stats_v0 https://portal.nersc.gov/project/m4134/FCN_weights_v0/stats_v0/global_stds.npy  
!wget -N -P FCN_weights_v0/stats_v0 https://portal.nersc.gov/project/m4134/FCN_weights_v0/stats_v0/land_sea_mask.npy
!wget -N -P FCN_weights_v0/stats_v0 https://portal.nersc.gov/project/m4134/FCN_weights_v0/stats_v0/latitude.npy    
!wget -N -P FCN_weights_v0/stats_v0 https://portal.nersc.gov/project/m4134/FCN_weights_v0/stats_v0/longitude.npy    
!wget -N -P FCN_weights_v0/stats_v0 https://portal.nersc.gov/project/m4134/FCN_weights_v0/stats_v0/time_means.npy   